# Automação Web e Busca de Informações com Python


In [5]:
#Instalar Pacotes
!pip install webdriver-manager
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime
from random import randint
from lxml import html
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
navegador = webdriver.Chrome(ChromeDriverManager().install())
navegador.quit()
from openpyxl import Workbook
!pip install schedule
import schedule

[WDM] - Downloading:  90%|██████████████████████████████████████████████████▌     | 5.84M/6.46M [00:00<00:00, 10.2MB/s]

In [6]:
#-----#------------------------#----------------------#_---------------------#-----------------
#Variáveis
#minutos de intervalo de rodagem de código
tempo_rodagem_codigo = 40
#número de paginas a fazer pesquisa
page_list = [1,2]

#nomes dos concorrentes
nomes_concorrentes = ['concorrente1','concorrente2']

#Filtros de modelos e precos. p é preco minimo; P é preco máximo (por modelo)
modelos_a_manter =  ['14', '13', '12','x','X']
p12 = 250
P12 = 550
p13 = 300
P13 = 8070
p14 = 640
P14 = 900

#Login OLX
username = 'joaocastelbranco44@gmail.com'
password = 'Comando44'
mensagem1 = 'Olá, estou interessado no Iphone.' 
mensagem2 = ' Poderia-me dizer se ainda está disponível? Obrigado.'

#---------#----------------------#--------------------#---------------------#-----------------

while(True):
    print('Início de código')
    st = time.time()
    #Fazer a Pesquisa
    #varáveis Xpath
    a = '//*[@id="root"]/div[1]/div[2]/form/div[5]/div/div[2]/div['
    b = ']/a/div/div/div[2]/div[1]/'
    f = ']/a/div/div/div[2]/div[2]/p'
    c = 'h6'
    d = 'p'
    e = ']/a'

    #criar listas
    descricao_lista = []
    modelo_lista = []
    preco_lista = []
    link_lista = []
    local_lista 
    data_lista = []
    data_envio_lista = []
    hora_envio_lista = []

    #abrir janela 
    navegador = webdriver.Chrome(ChromeDriverManager().install())
    navegador.maximize_window()

    #loop para cada pagina    
    for p in page_list: 
        navegador.get('https://www.olx.pt/d/telemoveis-e-tablets/telemoveis/benfica-lisboa/q-iphone/?search%5Bdist%5D=50&search%5Bfilter_float_price%3Afrom%5D=150&search%5Bfilter_enum_marca%5D%5B0%5D=iphone&page='+str(p))
        time.sleep(2)
    
        try :
            navegador.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        except NoSuchElementException:
                pass
    
        for i in range (2,60) :
            try:
                xpath_dinamico = a + str(i) + b
                preco=navegador.find_element_by_xpath(xpath_dinamico+d).text
                descricao=navegador.find_element_by_xpath(xpath_dinamico+c).text
                link_url=navegador.find_element_by_xpath(a + str(i)+e)
                link=link_url.get_attribute('href')
                local_data=navegador.find_element_by_xpath(a + str(i) + f).text    
        
            #modelo de iphone atraves da descricao
                try:
                    text=descricao.upper()
                    start = text.index('IPHONE')
                    end = text.index(' ',start+8)
                    modelo = text[start+1:end]
                    modelo = modelo[6:8]
                except ValueError :
                    pass
                try: 
                    local_data_split = local_data.split("-")
                except ValueError :
                    pass
            
                local = local_data_split[0]
                data = local_data_split[1]
                data = data[1:]
                hoje = datetime.today().strftime('%d-%m-%Y')
                data = data.replace('Hoje',hoje)
                data = data.replace('Para o topo','')
                preco = preco.replace('€','')
                preco = preco.replace(' \nNegociável','')
                preco = preco.replace('.','')                               
                preco = preco.replace(' ','')
                preco = preco.replace(',','.')
                preco = float(preco)
                    
                descricao_lista.append(descricao)
                modelo_lista.append(modelo)
                preco_lista.append(preco)
                link_lista.append(link)
                local_lista.append(local)
                data_lista.append(data)
            
            
            except NoSuchElementException:
                pass
            

    navegador.quit()
    print('Pesquisa feita')

    #------------#_--------------------------#-----------------#-------------

    #Criar Tabela da pesquisa
    lista_zip = list(zip(descricao_lista,
    modelo_lista,
    preco_lista,
    link_lista,
    local_lista,
    data_lista))

    #Create Table
    tabela = pd.DataFrame((lista_zip), columns=['descricao',
    'modelo',
    'preco',
    'link',
    'local',
    'data'])
    
    resultados_pesquisa = len(tabela.index)
    print('Pesquisa feita.',resultados_pesquisa,'resultados')
    tabela.to_excel(r'C:\Users\joaoc\Desktop\Fivver\Fivver OLX\tabela_olx.xlsx', index = False)

    #--------#-------------------------#_------------------------------------#_----------
    A_filtro_modelos_e_precos=len(tabela.index)
    #Filtro de modelos e preços
    tb = tabela[tabela['modelo'].isin(modelos_a_manter)]
    tb = tb.loc[~((tb['modelo'] == '13') & ((tb['preco'] > P13) | (tb['preco'] < p13))),:]
    tb = tb.loc[~((tb['modelo'] == '14') & ((tb['preco'] > P14) | (tb['preco'] < p14))),:]


    D_filtro_modelos_e_precos=len(tb.index)
    numero_filtrados_modelo_e_preco = A_filtro_modelos_e_precos - D_filtro_modelos_e_precos
    print('Filtragem de modelos e preços feita.',numero_filtrados_modelo_e_preco,'anuncios filtrados')

    #------------#_------------------------#--------------------------------#_----------
    
    tb_repetido_limpo = []
    #Eliminar anuncios ja enviados
    #Ler o documento de anuncios enviados para nao enviar de novo
    enviados_excluir = pd.read_excel('enviados_mensagem.xlsx')
    link_enviados = enviados_excluir['link'].values.tolist()
    #print (link_enviados)

    #Excluir os já enviados da lista a enviar
    tb_repetido_limpo = tb[tb.link.isin(link_enviados) == False].copy()
    
    D_filtro_enviados=len(tb_repetido_limpo.index)
    numero_filtrados_enviados = D_filtro_modelos_e_precos - D_filtro_enviados    
    print ('Filtragem de anuncios com mensagem já enviada feita.',numero_filtrados_enviados,'anuncios filtrados')
    
    #------------------------------##----------------------------------
    
    #Obter Lista de Anunciantes de anuncios
    URL_mensagem_lista = []
    anunciante_list = []
    i=0
    #ter a lista de URL
    for row in tb_repetido_limpo.index :
        URL_mensagem_lista.append(tb_repetido_limpo.iloc[i]['link'])  
        i=i+1

    #Abrir navegador
    navegador = webdriver.Chrome(ChromeDriverManager().install())
    navegador.maximize_window()
    #Loop obter lista anunciantes
    for URL in URL_mensagem_lista:
        navegador.get(URL)
        time.sleep(1)
        try :
            navegador.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        except NoSuchElementException:
                pass
        time.sleep(1)
        try :
            anunciante = navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[2]/div[1]/div[2]/div/a/div/div[2]/h4').text
        except NoSuchElementException:
            anunciante = navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[5]/div/div/div[1]/div/div[2]/h4').text
        except NoSuchElementException:
            anunciante = 'N/A'
            pass
        anunciante_list.append(anunciante)   

    navegador.quit()
    #Adicionar coluna anuncianta a tabela tb_repetido_limpo

    tb_repetido_limpo['anunciante'] = anunciante_list


    #excluir nome anunciantes
    tb_enviar = tb_repetido_limpo[tb_repetido_limpo.anunciante.isin(nomes_concorrentes) == False]
    
    D_filtro_concorrentes=len(tb_enviar.index)
    numero_filtrados_enviados = D_filtro_enviados - D_filtro_concorrentes
    print('Filtragem de anunciantes concorrentes feita.',numero_filtrados_enviados,'anuncios filtrados')

    #print(tb_nomes)
    #print(tb_enviar)

    #---------------------------#----------------------------#_-----------------------#_---------

    #ter a lista de URL para enviar
    URL_mensagem_lista_final = []
    j = 0
    for row in tb_enviar.index :
        #print(tb_enviar.iloc[j]['link'])
        URL_mensagem_lista_final.append(tb_enviar.iloc[j]['link']) 
        j=j+1

    #print(URL_mensagem_lista_final)

    #enviar mensagem a quem resta
    #abrir pagina
    navegador = webdriver.Chrome(ChromeDriverManager().install())
    navegador.maximize_window()
    k=1

    #para cada anuncio da tabela a enviar, enviar a mensagem
    for URL_enviar in URL_mensagem_lista_final:
        navegador.get(URL_enviar)
        #aceitar cookies
        time.sleep(2)
        try :
            navegador.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        except NoSuchElementException:
            pass
        #carregar botao enviar mensagem para fazer login
        try:
            time.sleep(2)
            time.sleep( randint ( 1, 2 ) )
            navegador.find_element_by_xpath('//*[@id="root"]/header/div/div/div[2]/a').click()
            time.sleep(2)
        except NoSuchElementException:
            pass        
            
        #fazer login
        try :
            time.sleep(1)
            time.sleep( randint ( 1, 3 ) )
            navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[1]/div/div/input').click()
            time.sleep( randint ( 1, 3 ) )
            navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[1]/div/div/input').send_keys(username)
            time.sleep( randint ( 1, 3 ) )
            navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[2]/div/div/div/input').click()
            time.sleep( randint ( 1, 3 ) )
            navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[2]/div/div/div/input').send_keys(password)
            time.sleep( randint ( 1, 2 ) )
            navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[2]/div/div/div/input').send_keys(Keys.ENTER)
            time.sleep(3)
        except NoSuchElementException:
            pass  
    #primeiro anuncio pos login
    #carregar botao enviar mensagem 
        time.sleep (3)
        navegador.execute_script("window.scrollTo(0, 1080)")
        time.sleep (1) 
        navegador.execute_script("window.scrollTo(0, 3000)")
        time.sleep (1)
        navegador.execute_script("window.scrollTo(0, 2000)")
        caixa = navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea')
        navegador.execute_script("arguments[0].scrollIntoView();", caixa)    
        time.sleep(1)
        try:
            navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea').click()
        except NoSuchElementException:
            pass  
        time.sleep( randint ( 1, 3 ) )
        navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea').send_keys(mensagem1)
        time.sleep( randint ( 1, 3 ) )
        navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea').send_keys(mensagem2)
        print('mensagem '+str(k)+' enviada')
        data_envio = datetime.today().strftime('%d-%m-%Y')
        hora_envio = datetime.today().strftime("%H:%M:%S")
        data_envio_lista.append(data_envio)
        hora_envio_lista.append(hora_envio)
        k=k+1
    tb_enviar['Data envio'] = data_envio_lista
    tb_enviar['Hora envio'] = hora_envio_lista
    print ('Mensagens enviadas')

    #--------------------------#_---------------------------------------#_-----------

    #Adicionar anuncios com mensagem enviada na lista enviados 
    from openpyxl import Workbook
    from openpyxl import load_workbook
    rows = tb_enviar.values.tolist()
    workbook = load_workbook(filename='enviados_mensagem.xlsx')
    sheet = workbook['Sheet1']
    for row in rows:
        sheet.append(row)
    workbook.save(filename='enviados_mensagem.xlsx')
    print ('Enviados adicionados a lista')
    et = time.time()
    elapsed_time = et - st
    tempo_codigo = elapsed_time /60
    print ('Programa rodado com sucesso.', k-1,'mensagens foram enviadas enviadas')
    print('Tempo de execução:', '%.2f' %tempo_codigo, 'minutos')
    print(tempo_rodagem_codigo,'minutos até à próxima')
    print('------#-----------#--------------#-----------------#------')
    navegador.quit()
    break
    #time.sleep(tempo_rodagem_codigo*60)


[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.46M/6.46M [00:09<00:00, 713kB/s]


Início de código
Pesquisa feita
Pesquisa feita. 52 resultados
Filtragem de modelos e preços feita. 34 anuncios filtrados
Filtragem de anuncios com mensagem já enviada feita. 3 anuncios filtrados
Filtragem de anunciantes concorrentes feita. 0 anuncios filtrados


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea"}
  (Session info: chrome=107.0.5304.108)


In [41]:
#enviar mensagem a quem resta
#abrir pagina
navegador = webdriver.Chrome(ChromeDriverManager().install())
navegador.maximize_window()
k=1

    #para cada anuncio da tabela a enviar, enviar a mensagem
for URL_enviar in URL_mensagem_lista_final:
    navegador.get(URL_enviar)
    #aceitar cookies
    time.sleep(2)
    try :
        navegador.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
    except NoSuchElementException:
        pass
        #carregar botao enviar mensagem para fazer login
#    try:
 #       time.sleep( randint ( 1, 2 ) )
  #      navegador.find_element_by_xpath('//*[@id="root"]/header/div/div/div[2]/a').click()
 #   except NoSuchElementException:
 #      pass        
    time.sleep(2)
 #----------#--------------------________________           
 #       #fazer login
 #   try :
  #      time.sleep(1)
   #     time.sleep( randint ( 1, 3 ) )
    #    navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[1]/div/div/input').click()
     #   time.sleep( randint ( 1, 3 ) )
      #  navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[1]/div/div/input').send_keys(username)
       # time.sleep( randint ( 1, 3 ) )
        #navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[2]/div/div/div/input').click()
        #time.sleep( randint ( 1, 3 ) )
  #      navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[2]/div/div/div/input').send_keys(password)
   #     time.sleep( randint ( 1, 2 ) )
    #    navegador.find_element_by_xpath('//*[@id="__next"]/div/div/div/div/main/div/div[2]/div/form/div[2]/div/div/div/input').send_keys(Keys.ENTER)
     #   time.sleep(1)
 #   except NoSuchElementException:
  #      pass  
    #primeiro anuncio pos login
    #carregar botao enviar mensagem 
   # time.sleep (3) 
    navegador.execute_script("window.scrollTo(0, 2100)") 
    time.sleep(1)
    navegador.execute_script("window.scrollTo(0, 1800)")
    time.sleep(1)
    
    #caixa = navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea')
    #navegador.execute_script("arguments[0].scrollIntoView();", caixa)
    time.sleep(1)
    try:
        navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div[1]').click()
    except NoSuchElementException:
        pass  
    time.sleep( randint ( 1, 3 ) )
    navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea').send_keys(mensagem1)
    time.sleep( randint ( 1, 3 ) )
    navegador.find_element_by_xpath('//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea').send_keys(mensagem2)
    print('mensagem '+str(k)+' enviada')
    data_envio = datetime.today().strftime('%d-%m-%Y')
    hora_envio = datetime.today().strftime("%H:%M:%S")
    data_envio_lista.append(data_envio)
    hora_envio_lista.append(hora_envio)
    k=k+1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/div[1]/div[3]/div[3]/div[1]/div[4]/section/div/div/form/div/div[1]/textarea"}
  (Session info: chrome=107.0.5304.108)
